In [22]:
import os
import random
from PIL import Image
from tqdm import tqdm

In [23]:
def place_chip_on_background(chip_path, background_path, output_path, scale_factor=0.5):
    # Load the chip and background images
    chip = Image.open(chip_path).convert("RGBA")
    background = Image.open(background_path).convert("RGBA")
    
    # Randomly rotate the chip
    angle = random.randint(0, 359)
    chip = chip.rotate(angle, expand=1)
    
    # Calculate new size for the chip, scaled by the given factor
    chip_width, chip_height = chip.size
    new_chip_size = (int(chip_width * scale_factor), int(chip_height * scale_factor))
    chip = chip.resize(new_chip_size, Image.Resampling.LANCZOS)
    
    # Update chip dimensions after resizing
    chip_width, chip_height = chip.size
    
    # Ensure the resized chip can fit into the background
    bg_width, bg_height = background.size
    if chip_width > bg_width or chip_height > bg_height:
        raise ValueError("The resized chip is still too large for the background.")
    
    # Randomly position the resized chip on the background
    max_x, max_y = bg_width - chip_width, bg_height - chip_height
    pos_x = random.randint(0, max_x)
    pos_y = random.randint(0, max_y)
    
    # Extract the alpha channel from the chip as the transparency mask
    mask = chip.split()[3]

    # Paste the resized chip onto the background using the mask
    background.paste(chip, (pos_x, pos_y), mask)
    
    # Save the combined image to the output path
    background.save(output_path)

In [24]:
def make_dataset(background_dir, chips_dir, output_dir, num_samples):
    # Lists to hold the file paths for chips and backgrounds
    chips = []
    backgrounds = []

    # Go through the backgrounds directory to collect background images
    for background_file in os.listdir(background_dir):
        backgrounds.append(os.path.join(background_dir, background_file))

    # Go through all the subdirectories in the chips directory to collect chip images
    for root, dirs, files in tqdm(os.walk(chips_dir), desc = "Loading Chips"):
        for chip_file in files:
            chips.append(os.path.join(root, chip_file))
    
    
    for i in tqdm(range(num_samples), desc = "Creating Dataset"):
        # Randomly select a chip and a background, and place the chip on the background
        selected_chip = random.choice(chips)
        selected_background = random.choice(backgrounds)

        output_image_path = os.path.join(output_dir, f"{i}.png")

        # Place the chip on the background
        place_chip_on_background(selected_chip, selected_background, output_image_path)

    

In [25]:
background_dir = "../Backgrounds"
chips_dir = "../Chips/CroppedChips"
output_dir = "../Dataset/AllChips/Singletons"
num_samples = 100


make_dataset(background_dir, chips_dir, output_dir, num_samples)

Loading Chips: 1it [00:00, 572.84it/s]
Creating Dataset: 100%|██████████████████████████████████████████████████████████████| 100/100 [00:08<00:00, 11.28it/s]
